In [1]:
import pyspark
import os

In [2]:
#only for local use
#in case problem with PYTHONHASHSEED occurs on cluster see https://www.thoughtvector.io/blog/python-3-on-spark-return-of-the-pythonhashseed/
#don't forget to unset on local env afterwards
os.environ["PYTHONHASHSEED"] = "42"

In [3]:
sc = pyspark.SparkContext("local[*]")

In [4]:
def read_graph(path_to_file):
    """ read graph in the moodle"""
    res = sc.textFile(path_to_file).map(lambda x : x.split("\t"))
    res = res.flatMap(lambda x : [(x[0], (x[1], int(x[2]))), (x[1], (x[0], int(x[3])))])
    return res

In [5]:
def read_twitter_graph(path_to_file):
    """ read the twitter graph"""
    res = sc.textFile(path_to_file).map(lambda x : tuple(x.split(" ")))
    return res

In [6]:
def read_generated_graph(path_to_file):
    res = sc.textFile(path_to_file).map(lambda x : (x.split("\t")[0], (x.split("\t")[1], int(x.split("\t")[2]))))
    return res

## graph non pondéré

In [7]:
#very small graph
directions = sc.parallelize([(1,2),(1,3),(2,4),(3,5),(4,5)])
begin = 1
end = 5
shortest_paths = sc.parallelize([(begin, (0,[]))])

In [8]:
directions = read_twitter_graph("twitter_combined.txt")

In [9]:
begin, end = '15519274', '309366491'
shortest_paths = sc.parallelize([(begin, (0,[]))])
early_stop = 15

In [10]:
%%time
i = 0
while True:
    res = shortest_paths.lookup(end)
    i+=1
    print(i)
    if not (res == [] and i < early_stop):
        break

    shortest_paths = shortest_paths.join(directions).map(lambda x : (x[1][1], (x[1][0][0] + 1 , x[1][0][1] + [x[0]])))

1


Py4JJavaError: An error occurred while calling o49.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/mnt/spark/twitter_combined.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:200)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrE

## graph pondéré

#### V1 (brute force)

In [ ]:
#connections are denoted (origin, (end, weight))

In [11]:
def compute_path(x):
    #x is the result of the join operation
    # the join should be in format (origin, ((weight_to_origin, path_to_origin, paths_visited_to_origin), (destination, weight_to_destination)))
    return (x[1][1][0], (x[1][0][0] + x[1][1][1], x[1][0][1] + [x[0]], {x[0]}))

In [12]:
def shortest_path_to_point(x,y):
    if x[0] <= y[0]:
        res = (x[0], x[1], x[2] | y[2])
    else:
        res = (y[0], y[1], x[2] | y[2])
    return res

In [19]:
#for a small graph test
directions = sc.parallelize([(1,(2,1)),(1,(3, 5)),(3,(2,2)),(2,(4,2)),(3,(5,2)),(4,(5,2))])
begin = 1
objective = 5
paths_to_objective = set(directions.map(lambda x : (x[1][0] , x[0])).filter(lambda x : x[0] == objective).values().collect())
shortest_paths = sc.parallelize([(begin, (0,[],set()))])

In [42]:
directions = read_generated_graph("../graph-data/weighted-graph-100.txt")
begin, objective = directions.keys().takeSample(False, 2)
paths_to_objective = set(directions.map(lambda x : (x[1][0] , x[0])).filter(lambda x : x[0] == objective).values().collect())
shortest_paths = sc.parallelize([(begin, (0,[],set()))])
early_stop = 15

In [43]:
%%time
i = 0
while True:
    #stopping criteria
    i += 1
    print(i)
    if i > early_stop :
        break
    path_found = shortest_paths.filter(lambda x : x[0] == objective).collect()
    print(path_found)
    print(shortest_paths.lookup(objective))
    if path_found != [] and path_found[0][1][2] == paths_to_objective :
        break
        
    #real thing
    shortest_paths = shortest_paths.join(directions).map(compute_path).union(shortest_paths)
    shortest_paths = shortest_paths.reduceByKey(shortest_path_to_point)

1
[]
[]
2
[]
[]
3
[('62', (21, ['47', '12'], {'12'}))]
[]
4
[('62', (21, ['47', '12'], {'12'}))]
[]
5
[('62', (21, ['47', '12'], {'12', '90'}))]
[]
6
[('62', (21, ['47', '12'], {'16', '12', '90'}))]
[]
CPU times: user 310 ms, sys: 150 ms, total: 460 ms
Wall time: 24.6 s


In [45]:
shortest_paths.filter(lambda x : x[0] == objective).collect()

[('62', (21, ['47', '12'], {'12', '16', '90'}))]

In [44]:
begin, objective

('47', '62')

#### V2

In [46]:
def compute_path(x):
    #x is the result of the join operation
    # the join should be in format (origin, ((weight_to_origin, path_to_origin, paths_visited_to_origin), (destination, weight_to_destination)))
    return (x[1][1][0], {
                         "weight_of_path" : x[1][0]["weight_of_path"] + x[1][1][1], 
                         "path" : x[1][0]["path"] + [x[0]], 
                         "explored_path" : {x[0]}
    })

In [47]:
def shortest_path_to_point(x,y):
    if x["weight_of_path"] <= y["weight_of_path"]:
        res = {"weight_of_path" : x["weight_of_path"], 
               "path" : x["path"], 
               "explored_path" : x["explored_path"] | y["explored_path"]}
    else:
        res = {"weight_of_path" : y["weight_of_path"], 
               "path" : y["path"], 
               "explored_path" : x["explored_path"] | y["explored_path"]}
    return res

In [23]:
# for small graph
begin = 1
objective = 5
early_stop = 1
directions = sc.parallelize([(1,(2,1)),(1,(3, 5)),(3,(2,2)),(2,(4,2)),(3,(5,2)),(4,(5,2))])
paths_to_objective = set(directions.map(lambda x : (x[1][0] , x[0])).lookup(5))
shortest_paths = sc.parallelize([(begin, {"weight_of_path" :0, "path" : [], "explored_path" : set()})])
points_to_drop = sc.broadcast(set())
continue_criteria = True

In [51]:
directions = read_generated_graph("../graph-data/weighted-graph-100.txt")
paths_to_objective = set(directions.map(lambda x : (x[1][0] , x[0])).filter(lambda x : x[0] == objective).lookup(objective))
shortest_paths = sc.parallelize([(begin, {"weight_of_path" :0, "path" : [], "explored_path" : set()})])
early_stop = 15
continue_criteria = True
points_to_drop = sc.broadcast(set())

In [52]:
%%time
i = 0
while continue_criteria:
    new_paths = shortest_paths.join(directions).map(compute_path)
    #print(new_paths.collect())
    try:
        min_new_paths =sc.broadcast(new_paths.map(lambda x : x[1]["weight_of_path"]).min())
        points_to_drop = sc.broadcast(set(shortest_paths.filter(lambda x : x[1]["weight_of_path"] < min_new_paths.value).keys().collect()) | points_to_drop.value)
    except ValueError:
        #if no new paths are detected
        min_new_paths = sc.broadcast(float("inf"))
    #print(min_new_paths.value)
    shortest_paths = new_paths.union(shortest_paths).reduceByKey(shortest_path_to_point).filter(lambda x : x[0] not in points_to_drop.value)
    directions = directions.filter(lambda x : x[0] not in points_to_drop.value and x[1][0] not in points_to_drop.value)
    #print(shortest_paths.collect())
    
    #stopping criteria
    continue_criteria = i < early_stop
    i +=1
    print(i)
    try: 
        #replace by lookup on real machine when hash problem is resolved
        path_to_objective = shortest_paths.filter(lambda x : x[0] == objective).collect()
        print(path_to_objective)
        continue_criteria = (not path_to_objective[0][1]["weight_of_path"] <= min_new_paths.value) and i < early_stop
    except IndexError:
        continue_criteria = min_new_paths.value != float("inf")
        pass
    except KeyError:
        continue_criteria = min_new_paths.value != float("inf")
        pass
        

1
[]
2
[('62', {'weight_of_path': 21, 'path': ['47', '12'], 'explored_path': {'12'}})]
3
[('62', {'weight_of_path': 21, 'path': ['47', '12'], 'explored_path': {'12'}})]
4
[('62', {'weight_of_path': 21, 'path': ['47', '12'], 'explored_path': {'12', '90'}})]
5
[('62', {'weight_of_path': 21, 'path': ['47', '12'], 'explored_path': {'16', '12', '90'}})]
6
[('62', {'weight_of_path': 21, 'path': ['47', '12'], 'explored_path': {'16', '12', '90'}})]
7
[('62', {'weight_of_path': 21, 'path': ['47', '12'], 'explored_path': {'16', '12', '90'}})]
8
[('62', {'weight_of_path': 21, 'path': ['47', '12'], 'explored_path': {'16', '12', '90'}})]
9
[('62', {'weight_of_path': 21, 'path': ['47', '12'], 'explored_path': {'16', '12', '90'}})]
10
[('62', {'weight_of_path': 21, 'path': ['47', '12'], 'explored_path': {'16', '12', '90'}})]
CPU times: user 6.44 s, sys: 4.01 s, total: 10.4 s
Wall time: 19min 28s


In [ ]:
new_paths.collect()